In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re
from openpyxl import Workbook, styles
from openpyxl.styles import PatternFill, Font
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import Rule

def lower_str(columns):
    newlist = list(map(lambda x: x.lower(), columns))
    return newlist

def check_bts(df_tw, bts_tw_columns, tw_index, status_col, df_msa, bts_msa_column, msa_index):
    #Nested Function to make conditional validations
    def cond_bts_check(bts_msa, tw_bts_sites):
        bts_out_tw=[]
        if sorted(bts_msa) != sorted(tw_bts_sites):
            for i in tw_bts_sites:
                if i not in bts_msa:
                    bts_out_tw.append(i)

        return bts_out_tw

    bts_msa = msa[msa[bts_msa_column]=='Yes']
    bts_msa = [str(i) for i in bts_msa[msa_index]]

    tw_bts_sites = df_tw[df_tw[bts_tw_columns]=='Yes']
    tw_bts_sites = [str(i) for i in tw_bts_sites[tw_index]]

    #return of datas
    filtered = df_tw[[tw_index, status_col]]
    bts_out_tw = cond_bts_check(bts_msa, tw_bts_sites)
    df = pd.DataFrame(bts_out_tw, columns=['New Sites'])
    df = pd.merge(df, filtered, how='left', left_on=['New Sites'], right_on=tw_index)
    df = df[['Bts_Sites_Out_UIS_File', status_col]]
    return df

def check_wip(df_tw,tw_index, wip_tw, tw_bts, df_msa, msa_index, wip_msa_col):

    #Nested Function to make conditional validations
    def cond_wip_check(wip_msa, tw_wip_sites):
        count_wip = 0
        wip_out_tw=[]
        if sorted(wip_msa) != sorted(tw_wip_sites):
            for i in tw_wip_sites:
                if i not in wip_msa:
                    count_wip += 1
                    wip_out_tw.append(i)

        return wip_out_tw

    wip_msa = df_msa[df_msa[wip_msa_col]=='Yes']
    wip_msa = [str(i) for i in wip_msa[msa_index]]

    tw_wip_sites = df_tw[df_tw[wip_tw]=='Yes']
    tw_wip_sites = [str(i) for i in tw_wip_sites[tw_index]]

    tw_wip_site_bts_flagged = df_tw[(df_tw[wip_tw]=='Yes')&(df_tw[tw_bts]=='Yes')]
    tw_wip_site_bts_flagged = tw_wip_site_bts_flagged[[tw_index, wip_tw, tw_bts]]

    wip_out_tw_list = cond_wip_check(wip_msa, tw_wip_sites)
    return wip_out_tw_list, tw_wip_site_bts_flagged
    """Reestrurar os script em PT, DE, CZ"""
    # Falta os outros países

def general_log_erros(df_list, sheet_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')   
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0)   
    writer.save() 



## Reading Excel File

In [ ]:
import pandas as pd 
import numpy as np
import re
def new_format(df, bts_col):
    new_format = []
    for i in df[bts_col]:
        if i == 'BTS':
            new_format.append('Yes')
        elif i == 'Legacy':
            new_format.append('No')
        else:
            new_format.append('')
    return new_format

def read_files(path, sheetname, n_skiprows, n_skip_columns, site_index, date_cols):
    import pandas as pd
    import numpy as np

    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    df = pd.read_excel(path, sheet_name = sheetname, skiprows = n_skiprows)

    df = df.iloc[:,n_skip_columns:]
    df = df.dropna(subset=[site_index], axis=0)
    df.columns = lower_str(list(df.columns))
    
    #Create empty columns
    for i in range(1, 19):
        df[f'column{i}'] = [np.nan for i in range(df.shape[0])]
    
    # Parsing Dates
    for date_col in date_cols:
        lista = []
        df[date_col] = df[date_col].replace([' '], '')
        df[date_col] = df[date_col].fillna('')
        for i in df[date_col]:
            dates_format = re.match(r'\d{2}\/\d{2}\/\d{4}', str(i)) or re.match(r'\d{2}\-\d{2}\-\d{4}', str(i))
            not_date_format = not re.match(r'\d{2}\/\d{2}\/\d{4}', str(i)) or not re.match(r'\d{2}\-\d{2}\-\d{4}', str(i))
            if i !='' and not_date_format:
                #print(towerdb[towerdb[date_col]==i][['identification - site key', date_col]])
                lista.append(f'{i:%d/%m/%Y}')
            
            elif dates_format:
                lista.append(i)
            else:
                lista.append('')
        df[date_col] = lista

    return df

"""Defining variables which is gonna be reusable in checks"""
tw_index = "identification - site key"
tw_doer_vf = 'msa - date of equipment removal (vf)'
tw_doer_wh = 'msa - date of equipment removal (wh)'
tw_status = 'other - status'
tw_bts = 'flag indicating bts site'
tw_bill = 'msa - billing trigger date'
"""tw_wip isn't have in GR so, billing trigger date should be in future"""
tw_decom = 'msa - date of decommissioning'
tw_amount = 'lease contract - current annual lease fee'
tw_critical = 'msa - critical site in excess of the 15.5% cap'

msa_index = 'identification - site key'
msa_bts = 'flag indicating bts site'
msa_doer_vf = 'msa - date of equipment removal (vf)'
msa_doer_wh = 'msa - date of equipment removal (wh)'
msa_status = 'Other - Status'
msa_bill = 'msa - billing trigger date'
#tw_wip isn't have in GR so, billing trigger date should be in future
msa_decom = 'msa - date of decommissioning'
msa_amount = 'lease contract - current annual lease fee'
msa_critical = 'msa - critical site in excess of the 15.5% cap'

guideline = ["towerdb owner", "identification - site key", "identification - code", "identification - fl",\
             "position - site name", "identification - other mno fl", "position - macro region", \
             "position - region", "position - municipality", "position - address", "position - latitude", \
             "position - longitude", "category - categorization by transmission sys (subcluster)", \
             "category - categorization by site type", "column1", "category - categorisation by inhabitants", \
             "other - ownership", "other - status", "column2", "column3", "column4", "column5", "column6", \
             "column7", "column8", "column9", "column10", "lease contract - current annual lease fee", \
             "lease contract - lease contract comment", "column11", "column12", "column13", "column14", \
             "tenants - anchor", "tenants - vf - ran sharing", "tenants - vf - passive", \
             "tenants - wind - ran sharing", "tenants - wind - passive", "tenants - cosmote - passive", \
             "tenants - ran sharing tenants", "tenants - passive tenants", "tenants - non-mno tenants", \
             "tenants - non-mno tenant name", "tenants - total tenants", "column15", \
             "ee - no. of 24hr generators", "ee - no. of standby generators", "kpi - sites", "kpi - tenants", \
             "msa - bts/replacement", "msa - bts commitment site", "column16", \
             "infrastructure type - technology", "infrastructure type - fibre / microwave", \
             "infrastructure type - floor space", "infrastructure type - tower height", "column17",\
             "notified-97 (gr use only)", "changes (gr use only)", "msa - billing trigger stop date", \
             "other - indoor vs outdoor", "skylon - category", "msa - standard configuration", \
             "msa - construction type", "msa - hub sites total mw diameter", "bp - consolidated classification",\
             "msa - critical site", "sensitive - due to", "sensitive - description", "sensitive - department", \
             "notified - department", "notified - due to", "msa - sensitive", "msa - notified", \
             "notified - status", "notified - details", "msa - rectifier", "msa - battery", "msa - aircon", \
             "msa - revenue generating", "msa - date of decommissioning", "flag indicating bts site",\
             "msa - billing trigger date", "msa - dg-shelter", "msa - critical site in excess of the 15.5% cap",\
             "msa - form of active sharing", "msa - start date for active sharing arrangement",\
             "msa - end date for active sharing arrangement", "column18", "msa - rfai date", \
             "msa - site acceptance date", "msa - sub-lease reason", "msa - sub-lease applies", \
             "msa - date of equipment removal (vf)", "msa - date of equipment removal (wh)"]
guideline = list(map(lambda x: re.sub(r'^\s+|\s+$', '', str(x.lower())), guideline))

date_columns = ['msa - start date for active sharing arrangement', 'msa - rfai date', 'msa - site acceptance date', 'msa - billing trigger date', 
            'msa - date of decommissioning', 'msa - date of equipment removal (vf)', 'msa - date of equipment removal (wh)', 'msa - billing trigger stop date',
            'msa - end date for active sharing arrangement']

path_tw = "/content/GR_TowerDB Jun'21 FINAL (Billing version).xlsx"
sheet_tw = 'GR_TowerDB'
towerdb = read_files(path_tw, sheet_tw, 0, 0, 'Identification - Site Key', date_columns)

towerdb.rename(columns={'msa - legacy/bts': "flag indicating bts site"}, inplace=True)

towerdb.columns = list(map(lambda x: re.sub(r'^\s+|\s+$', '', str(x.lower())), towerdb.columns.to_list()))
towerdb = towerdb[guideline]
towerdb['tenants - non-mno tenants'] = [int(i) if not pd.isnull(i) else '' for i in towerdb['tenants - non-mno tenants']]

towerdb[tw_bts] = new_format(towerdb, tw_bts)

towerdb.to_csv('/content/TowerDB_Greece_20210831.csv', index=False, encoding='iso-8859-1', errors='replace')

towerdb.head(3)


Csv(in month and true up) read


In [ ]:
from unidecode import unidecode
path_in_month = '/content/TowerDB_Greece_20210731_renan.csv'
past_tw = pd.read_csv(path_in_month, engine='python').fillna('')
past_tw.columns = lower_str(list(past_tw.columns))
#towerdb = towerdb.reindex(columns = lower_str(col_order))
past_tw['identification - site key'] = past_tw['identification - site key'].apply(unidecode)

In [ ]:
def check_columns_received(df, bill_cols):
    twdb_col = lower_str(list(df.columns))
    col_miss = [i for i in bill_cols if i not in twdb_col]
    """
    for i in bill_cols:
        if i not in twdb_col:
            col_miss.append(i)"""
    df_col_missing = pd.DataFrame(col_miss, columns=['Column(s) Missing'], index=range(len(col_miss)))
    return df_col_missing

!pip install unidecode
from unidecode import unidecode
path_msa = '/content/TowerDB_Greece_20210731.csv'
msa = pd.read_csv(path_msa, engine='python',encoding='latin1')
msa.columns = lower_str(list(msa.columns))
msa['identification - site key'] = msa['identification - site key'].apply(unidecode)
"""Fazer a organização das colunas somente com o ficheiro correto"""
"""Check Columns Received"""
df_cols = check_columns_received(towerdb,list(msa.columns))
df_cols

Check columns received looking for missing columns that is gonna be used in rating engine

Finding ? in site ID, if find change to A

In [ ]:
def replace_id(df, col_find, find_value, replace_to):
    lista = [re.sub(f'[{find_value}]',replace_to, str(x)) for x in df[col_find]]
    return lista

towerdb[tw_index] = replace_id(towerdb, tw_index.lower(), '?', 'A')


Aditional formating for legacy/bts sites 
ONLY xlsx File

Legacy to No

BTS to Yes

First Check - Dates Formats (dd/mm/YYYY) 

Columns: Billing trigger & DOER


In [ ]:
"""Verify before, cause you need to convert all values in cols for string format to check"""
def check_date_columns(df, df_index,status_col,columns, format):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    def count_duplicates(lista):
        count_dict = {}
        for entry in lista:
            if entry in count_dict.keys():
                count_dict[entry] += 1
            else:
                count_dict[entry] = 1
        
        duplicates = {}
        for k, v in count_dict.items():
            if v > 1:
                duplicates[k] = v
        return pd.DataFrame.from_dict(duplicates, orient='index', columns=['# of Duplicates'])

    df_dates = df[columns]
    df_dates['sites'] = df_dates[df_index]
    df_dates = df_dates.set_index('sites')

    df_de = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df_dates[df_index])
    if not df_duplicates.empty:
        df_dates.drop_duplicates(subset=[df_index], inplace=True)

    filtered = df[[df_index, status_col]]

    if format == 1:
        date_format = re.compile(r'\d{2}\-\d{2}\-\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                #print(type(df_dates.loc[de_site,de_column]))
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')
    else:
        date_format = re.compile(r'\d{2}\/\d{2}\/\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                            
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')
# Columns to functions
bill_col = [tw_index, tw_bill]
doer_cols = [tw_index, tw_doer_vf, tw_doer_wh]


actives = towerdb[towerdb[tw_status]=='In Service']
no_actives = towerdb[towerdb[tw_status]=='Dismantled / Active Lease']

 #Checking columns for errors
df_dates_bill_errors = check_date_columns(actives, tw_index, tw_status, bill_col, 2)
#NO errors

df_dates_doer_errors = check_date_columns(no_actives, tw_index,tw_status, doer_cols, 2) 
# Just Blank Values in DOER's Columns

print(df_dates_bill_errors)
print('\n')
print(df_dates_doer_errors)

Second Check - TW Amount value General (xxx.xx)

Column(s): Lease Contract - Current annual lease fee

In [ ]:
def check_amounts(df_check, df_index, status_col, columns, pattern=','):
    """
    Paramns:
    pattern: general (. to decimal), other (, to decimal)
    """

    df = df_check[columns]
    df['sites'] = df[df_index]
    df = df.set_index('sites')

    df_new = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df[df_index])
    if not df_duplicates.empty:
        df.drop_duplicates(subset=[df_index], inplace=True)
    
    filtered = df_check[[df_index, status_col]]

    for site in df[df_index]:
        for column in columns[1:]:
            if not str(df.loc[site,column]).__contains__(pattern):
                #print(df.loc[site,column])
                if df.loc[site,df_index] not in df_new.index:
                    df_new.loc[site,df_index] = df.loc[site,df_index]
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'
                else:
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'

    df_new = df_new.dropna(how='all', axis=1).fillna('Ok')       
    if not df_new.empty:
        df_new = pd.merge(df_new, filtered, how='left', left_on='identification - site key', right_on=tw_index)
        df_new = df_new.set_index(tw_index)
        df_new = df_new[[status_col]+ df_new.columns[:-1].tolist()]
        #df_new = df_new[['identification - site key', status_col]]
        return df_new
    else: 
        print('\nNo one columns with incorrect Amount format!\n')

amount_cols = [tw_index, tw_amount]
df_amount_errors = check_amounts(actives, tw_index, tw_status, amount_cols,'.')
df_amount_errors
#No one error


No one columns with incorrect Amount format!



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Thirth - Check Picklist values
Do this check only for sites In Service Status

Check On Air Picklist

In [ ]:
def check_picklist_v1(df,df_index,df_status, df_cols, picklist_dict):
    from collections import defaultdict

    df_picklist = df[df_cols].fillna('')
    df_picklist = df_picklist.to_dict(orient='list')

    new_dic = defaultdict(list)
    """for id in df_picklist[df_index]:
        if id not in new_dic.keys():
            new_dic[df_index].append(id)""" 
            
    for column in set(picklist_dict.keys()):
        for value in df_picklist[column]:
            if value not in set(picklist_dict[column]): 
                if pd.isnull(value) or pd.isna(value) or value=='' or value=='nan':
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorrect value: {value}')
            else:
                new_dic[column].append('Ok!')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok!', np.nan)
    df_errors = df_errors.dropna(how='all', axis=1)
    
    if not df_errors.empty:
        df = df[[df_index, df_status]]
        #df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
        df_errors[df_index] = df_picklist[df_index]
        df_errors = df_errors.set_index(df_index)
        df_errors = df_errors.dropna(how='all', axis=0)
        df_errors = df_errors.reset_index()
        df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
        df_errors = df_errors[[df_status]+ df_errors.columns[:-1].tolist()]
        df_errors = df_errors.set_index(df_index)
        return df_errors
    else:
        print('\nNo one Picklist Error Founded!\n')
        
picklist_tw = {
    'towerdb owner' : ['Vodafone','Wind'],
    'category - categorisation by inhabitants': ['Heavy Urban','Urban','Rural', 'Suburban'],
    'flag indicating bts site': ['Yes', 'No'],
    'other - ownership': ['Collocation_VF_Own','Collocation_WH_Own','Shared_VF_Own','Shared_WH_Own','Standalone'],
    'other - indoor vs outdoor': ['Indoor','Outdoor','N/A - No Active Equipment'],
    'msa - construction type': ['DAS','Rooftop','Greenfield'],
    'bp - consolidated classification': ['GBT','RTT','Transmission Site','Special','Transmission Site',\
                                         'Smaller Macro Site','Outdoor Small Cell','n/a - Data Unavailable'],
    'msa - rectifier': ['Yes', 'No'],
    'msa - battery': ['Yes', 'No'],
    'msa - aircon': ['No', 'Yes'],
    'msa - dg-shelter': ['Yes', 'No'],
    'msa - sub-lease applies': ['Yes', 'No']
}
picklist_cols = ['Identification - Site Key', 'TowerDB owner','Category - Categorisation by inhabitants',\
                'flag indicating bts site','Other - Ownership','Other - Indoor vs Outdoor','MSA - Construction Type',\
                'BP - Consolidated classification','MSA - Rectifier','MSA - Battery','MSA - Aircon',\
                'MSA - DG-Shelter','MSA - Sub-Lease Applies']

actives_1 = towerdb[towerdb[tw_status]=='In Service']
picklist_cols = lower_str(picklist_cols)
df_picklist = check_picklist_v1(actives_1, tw_index, tw_status, picklist_cols, picklist_tw)
df_picklist
#No Erros


No one Picklist Error Founded!



Check General Picklist

In [ ]:
general_pick = {
    'towerdb owner' : ['Vodafone','Wind'],
    'category - categorisation by inhabitants': ['Heavy Urban','Urban','Rural']}
general_picklist_cols = ['Identification - Site Key', 'TowerDB owner','Category - Categorisation by inhabitants']
df_picklist_general = check_picklist_v1(towerdb, tw_index,tw_status, lower_str(general_picklist_cols), general_pick)
df_picklist_general
#Error blank values

Fourth Check - Remove "N/A", "0" or "-" values


Fifth Check MoM Sites (BTS, decomissoned...)


In [ ]:
"""Falta Coluna de Flag Indicating BTS Site no twerdb recebido"""
def check_mom_bts(df_tw, tw_index,status_col, tw_col, df_msa, msa_index, msa_col):

    #c = country   
    msa_bts = df_msa[df_msa[msa_col]=='Yes']
    msa_bts_sites = [i for i in msa_bts[msa_index]]

    tw_bts = df_tw[df_tw[tw_col]=='Yes']
    tw_bts_sites = [i for i in tw_bts[tw_index]]

    out_tower_bts = [i for i in msa_bts_sites if i not in tw_bts_sites]
    filtered = tw_bts[tw_bts[tw_index].isin(out_tower_bts)]
    if not filtered.empty:
        return filtered[[tw_index,status_col, tw_col]]  
    else:
        print('\nNo Errors founded!\n')

df_mom_bts = check_mom_bts(actives, tw_index, tw_status, tw_bts, msa, msa_index, msa_bts)
# No one error df_mom_bts

df_mom_decom = check_mom_bts(actives, tw_index,tw_status, tw_decom, msa, msa_index, msa_decom)
# No one error df_mom_decom



No Errors founded!


No Errors founded!



Seventh Check If Bts sites has date in Billing Trigger Column 

In [ ]:
act = list(towerdb[towerdb[tw_status]=='In Service'][tw_index])
sites = [i for i in act if i not in uip_vf_sites]
len(sites)

4819

UIS for IN month check

In [ ]:
""" BTS sites"""
path_vf = '/content/UserInput_Greece_VF_20210731.xlsx'
path_wh = '/content/UserInput_Greece_WH_20210731.xlsx'
uip_names = ['Site_ID', 'BTS site applicable charge (Annual)', 'Energy: Diesel Charges']
uip_vf = pd.read_excel(path_vf ,sheet_name='SiteLevel',usecols=[0,1,4],skiprows=2)
uip_wh = pd.read_excel(path_wh ,sheet_name='SiteLevel',usecols=[0,1,4],skiprows=2)
uip = pd.concat([uip_vf, uip_wh], ignore_index=True)
uip.columns = uip_names
uip_vf.columns = uip_names
uip_wh.columns = uip_names
uip_vf_sites = [i for i in uip_vf['Site_ID']]
uip_wh_sites = [i for i in uip_wh['Site_ID']]


msa_sites = [i for i in msa[msa_index]]
tw_sites = [i for i in towerdb[tw_index]]
uip_sites = uip_vf_sites+uip_wh_sites

In [ ]:
""" BTS sites"""
path_vf = '/content/UserInput_Greece_VF_20210831.xlsx'
path_wh = '/content/UserInput_Greece_WH_20210831.xlsx'
uip_names = ['Site_ID', 'BTS site applicable charge (Annual)', 'Energy: Diesel Charges']
uip_vf = pd.read_excel(path_vf ,sheet_name='SiteLevel',usecols=[0,1,4],skiprows=2)
uip_wh = pd.read_excel(path_wh ,sheet_name='SiteLevel',usecols=[0,1,4],skiprows=2)
#uip = pd.concat([uip_vf, uip_wh], ignore_index=True)
#uip.columns = uip_names
uip_vf.columns = uip_names
uip_wh.columns = uip_names
uip_vf_sites = [i for i in uip_vf['Site_ID']]
uip_wh_sites = [i for i in uip_wh['Site_ID']]

msa_sites_vf = [i for i in msa[msa['towerdb owner']=='Vodafone'][tw_index]]
msa_sites_wh = [i for i in msa[msa['towerdb owner']=='Wind'][tw_index]]

tw_sites_vf = [i for i in towerdb[towerdb['towerdb owner']=='Vodafone'][tw_index]]
tw_sites_wh = [i for i in towerdb[towerdb['towerdb owner']=='Wind'][tw_index]]
#uip_sites = uip_vf_sites+uip_wh_sites


In [ ]:
len(msa_sites_vf)

2956

Vodafone

In [ ]:
def check_new_sites(df_towerdb, tw_index, bts_col, bill_col, status_col, msa_list, towerdb_list):
    
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    
    filtered = df_towerdb[[tw_index, status_col]]

    #Finding for ALL NEW SITES
    new_site = [i for i in towerdb_list if i not in msa_list]
    new_sites = pd.DataFrame(new_site, columns=['New_Sites'])
    new_sites = pd.merge(new_sites, filtered, how='left', left_on=['New_Sites'], right_on=tw_index)
    new_sites = new_sites[['New_Sites', status_col]]

    #create a copy to make index modifications
    df = df_towerdb.copy()

    #New columns with Site Codes
    df['Sites'] = df[tw_index]
    #defining site code to index
    df.set_index('Sites', inplace=True)

    #filtering by new sites
    df = df[df[tw_index].isin(new_site)]

    # Save information os sites with demerged date more than current date
    df[bill_col] = pd.to_datetime(df[bill_col], errors='coerce')
    df_bts_demerged = df[(df[bts_col]=='Yes') | (df[bill_col] > current_date)]
    
    #if not (new_sites.empty or bts_out_uis.empty or df_site_bts.empty):
    return new_sites, df_bts_demerged[[tw_index,status_col, bts_col, bill_col]]

#actives_vf = actives[actives["towerdb owner"]=='Vodafone']

new_sites_vf, df_bts_demerged_vf = check_new_sites(towerdb, tw_index, tw_bts,\
                                                    tw_bill,tw_status, msa_sites_vf, tw_sites_vf)

print(new_sites_vf)
print('\n')
print(df_bts_demerged_vf)
#No errors



Empty DataFrame
Columns: [New_Sites, other - status]
Index: []


Empty DataFrame
Columns: [identification - site key, other - status, flag indicating bts site, msa - billing trigger date]
Index: []


Wind

In [ ]:
actives_wh = actives[~(actives["towerdb owner"]=='Vodafone')]

new_siteswh, bts_sites_out_uis_wh = check_new_sites(actives_wh, tw_index, tw_bts, tw_bill, \
                                                    tw_status, msa_sites_wh, tw_sites_wh)
print(new_siteswh)
print('\n')
print(bts_sites_out_uis_wh)
# new = 13, out = 16, df_bts = 0

      New_Sites other - status
0  90642-Α64201            NaN
1  90750-Α75002     In Service
Empty DataFrame
Columns: [identification - site key, other - status, flag indicating bts site, msa - billing trigger date]
Index: []


Eighth - Check DOER 

In [ ]:
"""In services Checks"""
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    if not df_tw[date_col].empty:
        if t == 'doer':
            filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]') < current_date)]
            return filtered[[tw_index, status_col, date_col]] 
        else:
            #bill_dates = pd.to_datetime(df_tw[date_col], errors==coerrce)
            filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]').empty)]
            return filtered[[tw_index, status_col, date_col]] 
    else:
        print('Nothing wrong in services sites!')

in_service = 'In Service'
bill_col = [tw_index, tw_bill]
#doer_cols = [tw_index,  tw_doer_vf, tw_doer_wh]
actives_vf = actives[actives["towerdb owner"]=='Vodafone']
actives_wh = actives[~(actives["towerdb owner"]=='Vodafone')]

df_doer_vf_on = check_tw_bill_doer(actives_vf, tw_index, tw_doer_vf, tw_status, in_service, 'doer')

df_doer_wh_on = check_tw_bill_doer(actives_wh, tw_index, tw_doer_wh, tw_status, in_service, 'doer')
# NO errors in both files

print(df_doer_vf_on)
print('\n')
print(df_doer_wh_on)


In [ ]:
"""Planned Checks"""
def check_tw_doer_planned(df_tw, tw_index, doer_col,bill_col, status_col, dt_format="%Y-%m-%d"):
    """Only GR until now"""
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date, format=dt_format)
    if not df_tw[doer_col].empty:
        df_tw[bill_col] = pd.to_datetime(df_tw[bill_col],errors='coerce', format=dt_format)
        filtered = df_tw[(df_tw[status_col]=='Planned')&(not df_tw[doer_col].astype('datetime64[ns]').empty)&\
                         (df_tw[bill_col].astype('datetime64[ns]') < current_date)]
        return filtered[[tw_index, status_col, bill_col, doer_col]]  
    else:
        print('\nNothing wrong in services sites!\n')
# or towerdb[tw_status]=='Planned To be dismantled' or towerdb[tw_status]=='Planned To Be Dismantled')
twdb_vf = towerdb[(towerdb["towerdb owner"]=='Vodafone')&(towerdb[tw_status]=='Planned')]
twdb_wh = towerdb[(towerdb["towerdb owner"]=='Wind')&(towerdb[tw_status]=='Planned')]
format = "%Y-%m-%d"
df_doer_vf_plann = check_tw_doer_planned(twdb_vf, tw_index, tw_doer_vf,tw_bill, tw_status)
df_doer_wh_plann = check_tw_doer_planned(twdb_wh, tw_index, tw_doer_wh,tw_bill, tw_status)

print(df_doer_vf_plann)
print('\n')
print(df_doer_wh_plann)
#No errors in both


Nineth - Check Decomissioned Sites

In [ ]:
def check_decommissioned(df,df_index,status_col, decom_col, doer_col):
    #c = country.lower()
    filtered = df[(df[decom_col]=='Yes')&(df[doer_col]=="")]
    return filtered[[df_index,status_col, decom_col, doer_col]]
  
twdb_vf = towerdb[towerdb["towerdb owner"]=='Vodafone']
twdb_wh = towerdb[~(towerdb["towerdb owner"]=='Vodafone')]

df_doer_vf = check_decommissioned(twdb_vf, tw_index,tw_status, tw_decom, tw_doer_vf)
df_doer_wh = check_decommissioned(twdb_wh, tw_index,tw_status, tw_decom, tw_doer_wh)

print(df_doer_vf)
print('\n')
print(df_doer_wh)
#No ERRORS


Empty DataFrame
Columns: [identification - site key, other - status, msa - date of decommissioning, msa - date of equipment removal (vf)]
Index: []


Empty DataFrame
Columns: [identification - site key, other - status, msa - date of decommissioning, msa - date of equipment removal (wh)]
Index: []


Tenth - Check UIP Towerdb matches






## Vodafone

In [ ]:
def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, uip_sites):

    filtered = df_tw[[tw_index, status_tw_col]]
    #tw_active = df_tw[df_tw['Site Status']=='In Service']
    count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

    # check number of sites that are in uip file and doesn't have in df_tw
    in_service_uip_sites = [i for i in uip_sites if i not in count_tw_sites]
    in_service_uip_sites = pd.DataFrame(in_service_uip_sites,columns=['UIS Sites not active in TowerDB!'])
    in_service_uip_sites = pd.merge(in_service_uip_sites, filtered, how='left', left_on='UIS Sites not active in TowerDB!',\
                                    right_on=tw_index)
    in_service_uip_sites = in_service_uip_sites[['UIS Sites not active in TowerDB!', status_tw_col]]

    in_service_not_in_uis = [i for i in count_tw_sites if i not in uip_sites]
    in_service_not_in_uis = pd.DataFrame(in_service_not_in_uis,columns=['TowerDB Sites out UIS'])
    in_service_not_in_uis = pd.merge(in_service_not_in_uis, filtered, how='left', left_on='TowerDB Sites out UIS',\
                                    right_on=tw_index)
    in_service_not_in_uis = in_service_not_in_uis[['TowerDB Sites out UIS', status_tw_col]]
    
    #Check Decomissioned Sites
    tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
    decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIS File'])
    decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                    right_on=tw_index)
    decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col]]
    
    #Check BTS sites
    bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
    bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
    bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site out UIS File'])
    bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='BTS Site out UIS File',\
                                    right_on=tw_index)
    bts_sites_out_uip = bts_sites_out_uip[['BTS Site out UIS File', status_tw_col]]


    bts_tw_critical = [i for i in df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]]
    critical = [i for i in bts_tw_critical if i not in uip_critical]
    critical = pd.DataFrame(critical, columns=['Sites Beyond 10% out UIS'])
    critical = pd.merge(critical, filtered, how='left', left_on='Sites Beyond 10% out UIS',\
                                    right_on=tw_index)
    critical = critical[['Sites Beyond 10% out UIS', status_tw_col]]

    return in_service_uip_sites,in_service_not_in_uis, decomiss_sites_in_uip, bts_sites_out_uip, critical

tw_vf = towerdb[towerdb["towerdb owner"]=='Vodafone']
in_service_uip_sites_vf,in_service_not_in_uis_vf, decomiss_sites_in_uip_vf, bts_sites_out_uip_vf, critical_vf = check_uip_tw(tw_vf,tw_index, tw_status, \
                                                            tw_decom, tw_bts, tw_critical, uip_vf_sites)

print(in_service_uip_sites_vf)
print('\n')
print(in_service_not_in_uis_vf)
print('\n')
print(decomiss_sites_in_uip_vf)
print('\n')
print(bts_sites_out_uip_vf)
print('\n')
print(critical_vf)
# No one errors


## Wind

In [ ]:
tw_wh = towerdb[towerdb["towerdb owner"]=='Wind']
uis_sites_not_in_towerdbwh, in_service_not_in_uiswh, decomiss_sites_in_uiswh, bts_sites_out_uipwh, critical_wh = check_uip_tw(tw_wh,tw_index, tw_status, \
                                                            tw_decom, tw_bts,tw_critical, uip_wh_sites)
print(uis_sites_not_in_towerdbwh)
print('\n')
print(in_service_not_in_uiswh)
print('\n')
print(decomiss_sites_in_uiswh)
print('\n')
print(bts_sites_out_uipwh)
print('\n')
print(critical_wh)



In [ ]:
wh_special = tw_wh[(tw_wh['bp - consolidated classification']=='Special')&(tw_wh[tw_status]=='In Service')][tw_index].to_list()
wh_special
sp_wh_not_uis = [i for i in uip_wh_sites if i not in wh_special]
sp_wh_not_uis

vf_special = tw_vf[(tw_vf['bp - consolidated classification']=='Special')&(tw_vf[tw_status]=='In Service')][tw_index].to_list()

sp_vf_not_uis = [i for i in uip_vf_sites if i not in vf_special]
print(sp_vf_not_uis)


In [ ]:
def check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order):
    
    def check_uip_commercial_values(df_actual, df_before, merge_cols):
        df_atual = uip_comercial_actual
        df_ant = uip_comercial_before
        df_cross = pd.merge(df_actual, df_before, on=merge_cols, \
                            how='left', suffixes=('_actual', '_before'), indicator='Exist')
        df_cross['Exist'] = np.where(df_cross.Exist == 'both', 'Yes', 'No')
        df_cross['Equal Values'] = df_cross['Exist']
        
        return df_cross
    # Check for commercial Values into current UIP File and compare with UIP File before
    uip_comercial_actual = pd.read_excel(path_current,sheet_name='Commercial', names=col_names)
    #uip_comercial_actual = uip_comercial_actual[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_actual = replace_values(uip_comercial_actual, col_replace, value=0)

    uip_comercial_before = pd.read_excel(path_before,sheet_name='Commercial', names=col_names )
    #uip_comercial_before = uip_comercial_before[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_before = replace_values(uip_comercial_before, col_replace, value=0)

    df_commercial =  check_uip_commercial_values(uip_comercial_actual, uip_comercial_before, merge_cols)

    df_commercial = df_commercial.reindex(columns=col_order)
    df_commercial_diffs = df_commercial[df_commercial['Equal Values']=='No']
    return df_commercial_diffs

def check_diffs_v2(path_current, path_last):
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def highlight_diff(data, color='yellow'):
        attr = 'background-color: {}'.format(color)
        other = data.xs('Current', axis='columns', level=-1)
        return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                            index=data.index, columns=data.columns)

    _actual = pd.read_excel(path_current,sheet_name='Commercial').fillna('')

    _before = pd.read_excel(path_last,sheet_name='Commercial').fillna('')

    df_all = pd.concat([_actual, _before],axis='columns', keys=['Current', 'Last'])
    df_final = df_all.swaplevel(axis='columns')[_actual.columns[1:]]

    #df_final.style.apply(highlight_diff, axis=None)
    if not df_final.empty:
        return df_final[(_actual != _before).any(1)].style.apply(highlight_diff, axis=None)
    else:
        print('\nNo differences Founded!\n')

df_com_vf = check_diffs_v2('/content/UserInput_Greece_VF_20210831.xlsx', '/content/UserInput_Greece_VF_20210731.xlsx')

df_com_wh = check_diffs_v2('/content/UserInput_Greece_WH_20210831.xlsx', '/content/UserInput_Greece_WH_20210731.xlsx')


Eleventh - Check commercial values

Made Changes in comparisons

In [ ]:
#check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order)
path_vf_before = '/content/UserInput_Greece_VF_20210630.xlsx'
path_vf_curr = '/content/UserInput_Greece_WH_20210630.xlsx'
names = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2','Param3','Param4', 'Data_Type', 'Input_Value',\
        'Description/Instruction', 'Frequency of Update']
merge_cols = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2','Param3','Param4', 'Data_Type', \
              'Description/Instruction', 'Frequency of Update']
cols_ordered = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2','Param3','Param4','Data_Type','Input_Value_actual',\
                'Input_Value_before','Equal Values','Description/Instruction', 'Frequency of Update']
df_commercial_diffs_vf = check_commercial(path_vf, path_vf_before, 'Input_Value', names, merge_cols, cols_ordered)
# No differences
df_commercial_diffs_wh = check_commercial(path_wh, path_wh_before, 'Input_Value', names, merge_cols, cols_ordered)
# No differences

Commercial Vodafone

In [ ]:
path_vf_before = '/content/UserInput_Greece_VF_20210731.xlsx'
path_vf_curr = '/content/UserInput_Greece_VF_20210831.xlsx'
commercial_diffs = check_diffs_v2(path_vf_curr, path_vf_before, '')
commercial_diffs

Commercial Wind

In [ ]:
path_wh_before = '/content/UserInput_Greece_WH_20210731.xlsx'
path_wh_curr = '/content/UserInput_Greece_WH_20210831.xlsx'
commercial_wh = check_diffs_v2(path_vf_curr, path_vf_before, '')
commercial_wh

Check Diesel Charges

In [ ]:
uip_vf_legacy_sites = [i for i in uip_vf[uip_vf['BTS site applicable charge (Annual)'].notnull()]['Site_ID']]
uip_vf_diesel_sites = [i for i in uip_vf[uip_vf['Energy: Diesel Charges'].notnull()]['Site_ID']]

actives_tw_sites = [i for i in towerdb[(towerdb[tw_status]=='In service') | (towerdb[tw_status]=='Dismantled / Active Lease')][tw_index]]
print(len(uip_vf_legacy_sites))
print(len(uip_vf_diesel_sites))
#legacies_vf_not_active_or_dismantled = [i for i in uip_vf_legacy_sites if i not in actives_tw_sites]
print(len(legacies_vf_not_active_or_dismantled))

183

In [ ]:
vf_special

[]

In [ ]:
charge = uip_vf['BTS site applicable charge (Annual)'].notnull().isin(actives_tw_sites)
ind = uip_vf.index[charge==True]
ind

Int64Index([], dtype='int64')

In [ ]:
diesel = uip_vf['Energy: Diesel Charges'].notnull().isin(actives_tw_sites)
ind = uip_vf.index[diesel==True]
ind

Int64Index([], dtype='int64')

In [ ]:
log_dfs = {'Bill Dates Errors': df_dates_bill_errors, 
'DOER Dates Errors': df_dates_doer_errors, 
'General Picklist Errors': df_picklist_general
'Actives Picklist Errors': df_picklist,
'Month BTS Check': df_mom_bts,
'Month Decomissioned Check': df_mom_decom,
'VF - New Sites':new_sites_vf,
'VF - Demerged Date Error': df_bts_demerged_vf,
'WH - New Sites': new_siteswh,
'WH - Demerged Date Error': bts_sites_out_uis_wh,
'VF - DOER Dates Erros': df_doer_vf_on,
'WH - DOER Dates Erros': df_doer_wh_on,
'VF - DOER Plann Dates Error': df_doer_vf_plann,
'WH - DOER Plann Dates Error': df_doer_wh_plann,
'VF - Decom Dates Errors': df_doer_vf,
'WH - Decom Dates Errors': df_doer_wh,
'VF(UIS not active in Twdb)': in_service_uip_sites_vf,
'VF(twdb Sites out UIS)' : in_service_not_in_uis_vf,
'VF(Decom sites in UIS)' : decomiss_sites_in_uip_vf,
'VF(BTS\'s out UIS File)': bts_sites_out_uip_vf,
'VF(Critical out UIS)':critical_vf,
'WH(UIS not active in Twdb)': uis_sites_not_in_towerdbwh,
'WH(twdb Sites out UIS)' : in_service_not_in_uiswh,
'WH(Decom sites in UIS)' : decomiss_sites_in_uiswh,
'WH(BTS\'s out UIS File)': bts_sites_out_uipwh,
'WH(Critical out UIS)': critical_wh}

dfs = [in_service_not_in_uiswh, in_service_not_in_uis, uis_sites_not_in_towerdb]
names = ['WH In Serv Sites not UIS', 'VF In Serv Sites not UIS', 'VF UIS Sites not in TwDB']

general_log_erros(dfs, names, '/content/errors_wh_vf.xlsx')

In [ ]:
df_tw = [df_dates_doer_errors, df_picklist_general, bts_sites_out_uis_v, bts_sites_out_uis_wh]
sheetnames_tw = ['DOER Dates_Errors', 'Picklist Invalid Value General', 'VF_BTS_OUT_UIS', 'WH_BTS_OUT_UIS']

path_tw = '/content/TWDB_GR_In-Month_Errors_20210803.xlsx'
general_log_erros(df_tw, sheetnames_tw, path_tw)